# OCNC2023 STEPS Tutorial Exercises

In this notebook we will try to create a STEPS simulation script from scratch by modifying the examples given in the tutorial. Please follow the tutor's instruction step by step.

# Introduction

## Build a biochemical model

Here is the example of a 2nd order reaction $A+B\overset{k}{\rightarrow}C$, where the reaction constant $k$ is set to 200 /uM.s:

```python
import steps.interface

# Import biochemical model module
from steps.model import *

# Create model container
mdl = Model()

# Create reaction manager
r = ReactionManager()

# Using this model, declare species and reactions
with mdl:
    # Create chemical species
    SA, SB, SC  = Species.Create()

    # Create reaction set container
    vsys = VolumeSystem.Create()

    with vsys:
        # Create reaction
        # SA + SB - > SC with rate 200 /uM.s
        SA + SB >r[1]> SC
        r[1].K = 200e6
```
Bidirectional reactions can also be written and require two rate constants (the forward one and the backward one):
```python
with mdl:
    # ...
    with vsys:
        SA + SB <r[1]> SC
        r[1].K = 200e6, 10
```

Instead of using `>r[1]>` in between reaction sides, we used `<r[1]>`, and we gave two rate constants when setting `r[1].K`. 

The first one corresponds to the forward rate (the `SA + SB -> SC` reaction) while the second one corresponds to the backward rate (the `SC -> SA + SB` reaction)

It is thus equivalent to:
```python
with mdl:
    # ...
    with vsys:
        SA + SB >r[1]> SC
        r[1].K = 200e6
        
        SC >r[2]> SA + SB
        r[2].K = 10
```

# Exercise 1: Create a kinase reaction model in STEPS

Modify the script below for this kinase reaction system:

$MEKp+ERK\underset{0.6}{\overset{16.2*10^{6}}{\rightleftarrows}}MEKpERK\overset{0.15}{\rightarrow}MEKp+ERKp$

**Hint**: Break it down in to these reactions

1: $MEKp+ERK\underset{0.6}{\overset{16.2*10^{6}}{\rightleftarrows}}MEKpERK$

3: $MEKpERK\overset{0.15}{\rightarrow}MEKp+ERKp$

In [ ]:
# Uncomment the line below and run the cell to load the answer
#%load ./answers/ans1.py

import steps.interface

# Import biochemical model module
from steps.model import *

# Create model container
mdl = Model()

with mdl:
    # Create chemical species
    MEKp, ERK, MEKpERK, ERKp = Species.Create()

    # Create reaction set container (volume system)
    vsys = VolumeSystem.Create()

    with vsys:
        r = ReactionManager()

        # MEKp + ERK <-> MEKpERK, rate constants 16.2*1e6 (forward) and 0.6 (backward)

        # MEKpERK -> MEKp + ERKp, rate constant 0.15

# Check that everything is in the model
print(*mdl.ALL())

## Setting up geometry

You can easily setup the geometry of a well-mixed model by creating a compartment and providing the volume system it is associated with as well as its volume. For example:
```python
# Import geometry module
from steps.geom import *

# Create well-mixed geometry container
wmgeom = Geometry()

# Using this geometry, declare compartments
with wmgeom:
    # Create cytosol compartment, associate it with vsys, give volume in m^3
    cyt = Compartment.Create(vsys, vol=1e-18)
```

## Creating a random number generator

You can use the following code to create a random number generator for the simulation, currently available generators are "mt19937" and "r123":
```python
# Import random number generator module
from steps.rng import *

# Create random number generator, with buffer size as 256 and starting seed 899
rng = RNG('mt19937', 256, 899)

# Could use time to get random seed
# import time
# rng = RNG('mt19937', 256, int(time.time()))
```

# Exercise 2: Create the geometry and random number generator for the kinase reaction model
    
Let's continue our kinase reaction simulation script, here are the tasks:
1. Create a compartment of $0.1um^{3}$ (note that STEPS uses S.I units)
2. Associate the compartment with the volume system we've previously created
3. Create a "r123" random number generator and initialize it with with some seed

In [ ]:
# Uncomment the line below and run the cell to load the answer
#%load ./answers/ans2.py

from steps.geom import *
from steps.rng import *

# Create a well-mixed geometry container

# Create a compartment of  0.1um^3 and associate it to volume system 'vsys'

# Create a 'r123' random number generator


## Creating and initializing a simulation

For well-mixed simulation, we create a simulation using the "Wmdirect" solver and initialize it by adding molecules to the compartment:
```python
# Import simulation module
from steps.sim import *

# Create well-mixed stochastic simulation object
sim_wm = Simulation('Wmdirect', mdl, wmgeom, rng)

# Signalize the start of a new run
sim_wm.newRun()

# Initialize the number of 'SA' molecules to 10
sim_wm.cyt.SA.Count = 10

# Or you can set the concentration (M), as for 'SB'
sim_wm.cyt.SB.Conc = 3.32e-08
```


## Running the solver and gathering simulation data

After that we can run the solver until it reaches a specific time point, say 0.1 second. You can explicitely gather simulation data such as molecule counts using a similar syntax:
```python
# Run simulation for 0.1s
sim_wm.run(0.1)

# Return the number of SA molecules in compartment cyt
sim_wm.cyt.SA.Count
```
---
```
10
```
---

## Gathering simulation data automatically

In practice, it is often more convenient to let STEPS know which data you want to save and let it handle the creation of data structures and the recording of values automatically. This needs to be done before any runs. For example, if we wanted to record the counts of species `SA`, `SB`, and `SC`, we could write:
```python
from steps.sim import *
sim_wm = Simulation('Wmdirect', mdl, wmgeom, rng)

# Create a result selector associated to the sim_wmulation
rs = ResultSelector(sim_wm)

# Specify which values should be saved, the << operator is used to concatenate values
counts_wm = rs.cyt.SA.Count << rs.cyt.SB.Count << rs.cyt.SC.Count

# Save these values every 0.001s
sim_wm.toSave(counts_wm, dt=0.001)

sim_wm.newRun()
sim_wm.cyt.SA.Count = 10
sim_wm.cyt.SB.Conc = 3.32e-08
sim_wm.run(0.1)

# Retrieve the time points for run 0
counts_wm.time[0]
```
---
```
[0.    0.001 0.002 0.003 0.004 0.005 0.006 0.007 0.008 0.009 0.01  0.011
 0.012 0.013 0.014 0.015 0.016 0.017 0.018 0.019 0.02  0.021 0.022 0.023
 ...
 0.084 0.085 0.086 0.087 0.088 0.089 0.09  0.091 0.092 0.093 0.094 0.095
 0.096 0.097 0.098 0.099 0.1  ]
```
---
```python
# Retrieve the saved data for run 0
counts_wm.data[0]
```
---
```
[[10. 20.  0.]
 [10. 20.  0.]
 [10. 20.  0.]
 ...
 [ 7. 17.  3.]
 [ 7. 17.  3.]
 [ 7. 17.  3.]]
```
---
The first column correspond to `rs.cyt.SA.Count` and for each time in `counts_wm.time[0]` there is a corresponding row in `counts_wm.data[0]`
We can also see which values have been saved with:
```python
counts_wm.labels
```
---
```
['cyt.SA.Count', 'cyt.SB.Count', 'cyt.SC.Count']
```
---

Alternatively, instead of listing every species separately we can use:
```python
counts_wm = rs.cyt.ALL(Species).Count
```
which is equivalent to the previous one. We can also record a subset of the existing species with e.g.:
```python
counts_wm = rs.cyt.LIST(SA, SC).Count
```

# Exercise 3: Run your kinase model in STEPS
Here are the tasks:
1. Create a simulation using the "Wmdirect" solver
2. Set-up automatic recording of the concentration of each molecule every 0.01 seconds
3. Start a new run and set the initial conditions:
    * MEKp = 1uM
    * ERK = 1.5uM
4. Run the simulation for 30 seconds

In [ ]:
# Uncomment the line below and run the cell to load the answer
#%load ./answers/ans3.py

from steps.sim import *
from steps.saving import *

# Create a simulation using the 'Wmdirect' solver

# Set-up automatic recording of the concentration of each molecule every 0.01 seconds

# Start a new run and set the initial conditions

# Run the simulation for 30 seconds


## Visuzalizing simulation data

Visualization is often needed to analyze the behavior of the simulation, in this tutorial, we will use Matplotlib to plot the data:
```python
import matplotlib.pyplot as plt
```
By default, the automatically saved data can be plotted with one line e.g.:
```python
plt.plot(counts_wm.time[0], counts_wm.data[0])
```
And the corresponding labels can be added with:
```python
plt.legend(counts_wm.labels)
```

# Execise 4: Plot the results of the kinase simulation

Modify the following code to plot your data:

In [ ]:
# Uncomment the line below and run the cell to load the answer
#%load ./answers/ans4.py

%matplotlib inline
import matplotlib.pyplot as plt

# Uncomment and replace with the name of your results object
# plt.plot(___.time[0], ___.data[0])
plt.ylabel('Number of molecules')
plt.xlabel('Time [s]')
# Uncomment and replace with the name of your results object
# plt.legend(___.labels)
plt.show()


## From well-mixed simulation to spatial simulation

To convert a well-mixed simulation to a spatial one, here are the basic steps:
1. Add diffusion rules for every diffusive species in the biochemical model.
2. Change the well-mixed geometry to a tetrahedral mesh.
3. Change the solver to "Tetexact".

First, let's see how to add diffusion rules in our example well-mixed model:

```python
# ...
with mdl:
    SA, SB, SC  = Species.Create()
    vsys = VolumeSystem.Create()
    with vsys:
        SA + SB >r[1]> SC
        r[1].K = 200e6
        
        # Create diffusion rules for species SA, SB, and SC
        diff_SA = Diffusion.Create(SA, 0.02e-9)
        diff_SB = Diffusion.Create(SB, 0.02e-9)
        # First argument gives the affected species, second argument gives the diffusion coefficient in m^2/s
        diff_SC = Diffusion.Create(SC, 0.02e-9)
```

We would then need to import a tetrahedral mesh to replace the well-mixed geometry we used so far.

```python
from steps.geom import *

# Import the mesh and scale it so that dimensions are in meters
mesh = TetMesh.LoadGmsh('meshes/sp_0.1v_1046.msh', 1e-6)

with mesh:
    # Create the compartment by providing a list of all tetrahedrons
    cyt = Compartment.Create(mesh.tets, vsys)
```

Finally, we would need to replace the "Wmdirect" solver with the spatial "Tetexact" solver:
```python
from steps.sim import *

# Create a spatial simulation with the Tetexact solver
sim_tet = Simulation('Tetexact', mdl, mesh, rng)
```
The rest of the script would be identical.

# Exercise 5: Create a spatial kinase simulation

Before this execise, let's make sure the tetrahedral mesh is ready by visualizing it with the code below.

In [ ]:
import pyvista as pv
plotter = pv.Plotter()
mesh = pv.read("meshes/spine.msh")
plotter.add_mesh(mesh, style="wireframe", show_scalar_bar=False)
plotter.show_bounds(grid='front', location='outer', all_edges=True)
plotter.show(jupyter_backend='panel')

Let's now convert the below well-mixed kinase model to a spatial one, here are the tasks:

1. Add diffusion rules with the following diffusion coefficients:
    * MEKp = 30e-12 $m^2/s$
    * ERK = 30e-12 $m^2/s$
    * MEKpERK = 10e-12 $m^2/s$
2. Replace the geometry to use mesh 'meshes/spine.msh'
3. Create a mesh-based compartment
4. Change the solver to Tetexact
5. Uncomment data saving code from previous cells
4. Run the simulation again

In [ ]:
# Uncomment the line below and run the cell to load the answer
#%load ./answers/ans5.py

# Create a new Model
mdl_tet = Model()

with mdl_tet:
    MEKp, ERK, MEKpERK, ERKp = Species.Create()
    vsys = VolumeSystem.Create()
    with vsys:
        r = ReactionManager()
        # Previously declared reactions
        MEKp + ERK <r[1]> MEKpERK
        r[1].K = 16.2*1e6, 0.6
        MEKpERK >r[1]> MEKp + ERKp
        r[1].K = 0.15

        # Add diffusion rules

# Load the 'meshes/spine.msh' mesh and create a cyt compartment
# The mesh is in micrometers and should be rescaled to meters

# Create a new simulation that uses the 'Tetexact' solver 
# sim_tet = ...

# Uncomment the following code:

# rs = ResultSelector(sim_tet)
# counts_tet = rs.cyt.ALL(Species).Count
# sim_tet.toSave(counts_tet, dt=0.01)

# sim_tet.newRun()

# sim_tet.cyt.MEKp.Conc = 1e-6
# sim_tet.cyt.ERK.Conc = 1.5e-6

# sim_tet.run(30)

# plt.plot(counts_tet.time[0], counts_tet.data[0])
# plt.ylabel('Number of molecules')
# plt.xlabel('Time [s]')
# plt.legend(counts_tet.labels)
# plt.show()
